## TripleTen Code Jam 2023
**Team: Santa's Coders**
- Marco Fernstaedt
- Veronica Steele
- Andrew Kwon

In this project, our team designed a delivery route application where users can select a start location and up to seven destinations. This project applies various methods to solving/estimating a small scale adaptation of the traveling salesman problem, or shortest Hamiltonian cycle.

### Packages and Libraries

In [1]:
import pandas as pd
import random
import time
import math

from itertools import permutations
from geopy import distance

### Data Loading and Processing

The dataset is derived from the 2020 US census report and contains information such as city name, state, counties, and population. The most important information we'll need for our route optimization are the geodetic coordinates for each location.

In [2]:
cities = pd.read_csv('datasets/us_cities.csv', encoding = 'ISO-8859-1')

display(cities.sample(10))
cities.info()

,City,State,Type,Counties,Population,Latitude,Longitude
10262,Stratford,OK,Town,Garvin,1405,34.795,-96.960
17976,Grenora,ND,City,Williams,221,48.620,-103.937
18578,Fitzhugh,OK,Town,Pontotoc,183,34.658,-96.767
5640,Lindstrom,MN,City,Chisago,4888,45.405,-92.865
779,Kannapolis,NC,City,Rowan;Cabarrus,53114,35.489,-80.628
18320,Trenton,SC,Town,Edgefield,200,33.741,-81.841
979,Woonsocket,RI,City,Providence,43240,42.000,-71.516
18840,Centerville,MO,City,Reynolds,167,37.436,-90.960
819,East Honolulu,HI,Census-Designated Place,Honolulu,50922,21.286,-157.707
18065,Frederick,SD,Town,Brown,215,45.832,-98.507


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21397 entries, 0 to 21396
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        21397 non-null  object 
 1   State       21397 non-null  object 
 2   Type        21397 non-null  object 
 3   Counties    21397 non-null  object 
 4   Population  21397 non-null  int64  
 5   Latitude    21397 non-null  float64
 6   Longitude   21397 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.1+ MB


The dataset appears to be intact; there are 21,397 entries with no missing values. The data types for each column also appear to be correct. In order to simplify the selectable destinations along our route, we'll limit the dataset to the most popular US cities. As such, we'll only include cities that have a population of 400,000 people or greater. Additionally, we'll drop the columns we will not be using -- counties and type.

In [3]:
pop_cities = cities.drop(['Counties', 'Type'], axis=1)
pop_cities = pop_cities[pop_cities['Population'] >= 4e5]

display(pop_cities.sample(10))
print(pop_cities.shape)

,City,State,Population,Latitude,Longitude
43,Miami,FL,442241,25.774,-80.192
27,Memphis,TN,633104,35.146,-90.052
15,Charlotte,NC,874579,35.227,-80.843
25,Las Vegas,NV,641903,36.171,-115.144
12,Fort Worth,TX,918915,32.754,-97.331
17,Seattle,WA,737015,47.607,-122.338
10,Austin,TX,961855,30.268,-97.743
41,Long Beach,CA,466742,33.768,-118.192
33,Fresno,CA,542107,36.737,-119.789
22,El Paso,TX,678815,31.760,-106.488


(48, 5)


As a result, we have 48 cities from which users can choose for their route.

### Export Subset Data

Export to facilitate webapp loading

In [4]:
# Remove comment if export required
# pop_cities.to_csv('datasets\cities_final.csv')

### Route Algorithms

To illustrate optimization improvements, we'll compare the following search path algorithms:  
- Random path (baseline)
- Greedy
- K-optimal Tours (Heuristic)
- Brute Force

**Functions**

<code>get_city_data(cities)</code>  
Parameters:  
- *cities*: List of city names (dtype string) of start node and route destinations

Returns:
- DataFrame slice of city name, state, population, latitude and longitiude

In [5]:
def get_city_data(city_list):
    return pop_cities.query('City in @city_list')

<code>create_map(df_cities)</code>  
Parameters:  
- *df_cities*: DataFrame slice of name, state, population, latitude and longitiude for start and destination cities

Returns:
- *edges*: Dictionary object of graph edges (all city pairs)
- *weights*: Dictionary objects containing the edge weights between each city pair
- *coords*: Dictionary object of each city (dtype string) and their location in latitude/longitude (tuple of float)

In [6]:
def create_map(df_cities):
    num_cities = len(df_cities)
    edges = {}
    weights = {}
    coords = {}
    
    for i in range(num_cities):
        city_a = df_cities.iloc[i]
        edges[city_a['City']] = []
        coords[city_a['City']] = (city_a['Latitude'], city_a['Longitude'])
        for j in range(num_cities):
            if i != j:
                city_b = df_cities.iloc[j]
                edges[city_a['City']].append(city_b['City'])
                weights[(city_a['City'], city_b['City'])] = distance.distance((city_a['Latitude'], city_a['Longitude']), (city_b['Latitude'], city_b['Longitude'])).nm

    return edges, weights, coords

<code>get_random_rte(src, nodes, weights)</code>  
Parameters:  
- *src*: Start and end location (dtype string)
- *nodes*: List of destination cities (dtype string)
- *weights*: Dictionary object of edge weights between cities (distance in nautical miles)

Returns:  
- *route*: Randomly generated route, starting and ending with the source node (dtype list)
- *cost*: The tour cost for the route in nautical miles (dtype float)

In [7]:
def get_random_rte(src, nodes, weights):
    random.shuffle(nodes)
    route = [src] + nodes + [src]
    
    curr = route[0]
    cost = 0
    for i in range(1, len(route)):
        next = route[i]
        cost += weights[(curr, next)]
        curr = next
    
    return route, cost

<code>get_greedy_rte(src, edges, weights)</code>  
Parameters:  
- *src*: Start and end location (dtype string)
- *edges*: Dictionary object of each city pair representing graph edges
- *weights*: Dictionary object of edge weights between cities (distance in nautical miles)

Returns:  
- *route*: Route generated by selecting the next closest city from each node, starting at the source (dtpe list)
- *cost*: The tour cost for the route in nautical miles (dtype float)

In [8]:
def get_greedy_rte(src, edges, weights):
    visited = set()
    v = src

    route = []
    cost = 0
    while True:
        visited.add(v)
        route.append(v)
        min_cost = math.inf
        for u in edges[v]:
            if (u not in visited) and (weights[(v, u)] < min_cost):
                min_cost = weights[(v, u)]
                next_node = u
        if min_cost < math.inf:
            v = next_node
            cost += min_cost
        else:
            break
    route.append(src)
    cost += weights[(v, src)]

    return route, cost

<code>get_route_cost(route)</code>
Parameters:
- *route*: Hamiltonian route (dtype list)

Returns:
- *cost*: Tour cost in total distance (dtype float)

In [9]:
def get_route_cost(route, weights):
    curr = route[0]
    cost = 0
    for i in range(1, len(route)):
        next = route[i]
        cost += weights[(curr, next)]
        curr = next
    
    return cost

<code>swap_edges(route, v, u)</code>
Parameters:  
- *route*: Current route to be optimized
- *v*: Index of first node to be swapped in new route
- *u*: Index of second node to be swapped in new route

Returns:
- *new_route*: New route with 2-opt swapped nodes

In [10]:
def swap_edges(route, v, u):
    new_route = []
    for i in range(v+1):
        new_route.append(route[i])
    for j in range(u, v, -1):
        new_route.append(route[j])
    for k in range(u+1, len(route)):
        new_route.append(route[k])
    
    return new_route

<code>get_2_opt_rte(src, route, weights)</code>  
Parameters:  
- *route*: Input route, arbitrarily generated (dtype list)
- *weights*: Dictionary object of edge weights between cities (distance in nautical miles)

Returns:  
- *route*: Heuristically generated route using 2-opt swapping (dtype list)
- *cost*: The tour cost for the route in nautical miles (dtype float)

In [11]:
def get_2_opt_rte(route, weights):
    curr_route = route
    best_distance = get_route_cost(curr_route, weights)
    can_improve = True
    while can_improve:
        can_improve = False
        for i in range(1, len(route)-2):
            for j in range(i+1, len(route)-1):
                new_route = swap_edges(curr_route, i, j)
                new_distance = get_route_cost(new_route, weights)
                if new_distance < best_distance:
                    curr_route = new_route
                    best_distance = new_distance
                    can_improve = True
    
    return curr_route, best_distance

<code>get_exact_rte(src, nodes, weights)</code>  
Parameters:  
- *src*: Start and end location (dtype string)
- *nodes*: List of destination cities (dtype string)
- *weights*: Dictionary object of edge weights between cities (distance in nautical miles)

Returns:  
- *route*: Route of exact shortest route, calculated via brute force (dtype list)
- *cost*: The tour cost for the route in nautical miles (dtype float)route in nautical miles (dtype float)

In [12]:
def get_exact_rte(src, nodes, weights):
    all_rtes = list(permutations(nodes))
    best_rte = None
    min_cost = math.inf

    for rte in all_rtes:
        route = [src] + list(rte) + [src]
        curr = route[0]
        cost = 0

        for i in range(1, len(route)):
            next = route[i]
            cost += weights[(curr, next)]
            curr = next
        
        if cost < min_cost:
            min_cost = cost
            best_rte = route
    
    return best_rte, min_cost

**Test Routes**
- Start: Phoenix
- Destinations: Random sample from filtered DataFrame

In [13]:
home = 'Phoenix'
dest_cities = pop_cities['City'].sample(7, random_state=42).to_list()
print(f'Home: {home}')
print(f'Cities: {dest_cities}')

Home: Phoenix
Cities: ['Memphis', 'Raleigh', 'Detroit', 'Miami', 'Portland', 'Atlanta', 'Fort Worth']


Initalize parameters and calculate graph edges and weights.

In [14]:
city_list = [home] + dest_cities
df_destinations = get_city_data(city_list)
edges, weights, coords = create_map(df_destinations)

**Random Route**

In [15]:
t0 = time.time()
route, total_dist = get_random_rte(home, dest_cities, weights)
t1 = time.time()

print(f'Random Route: {route}')
print(f'Total Distance: {"{:.1f}".format(total_dist)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

Random Route: ['Phoenix', 'Portland', 'Miami', 'Atlanta', 'Detroit', 'Fort Worth', 'Memphis', 'Raleigh', 'Phoenix']
Total Distance: 7763.5 nm
Calculation Time: 0.0 seconds


**Greedy Route**

For each destination on the route, choose the next closest city to visit, then return to the home city.

In [16]:
t0 = time.time()
route, total_dist = get_greedy_rte(home, edges, weights)
t1 = time.time()

print(f'Greedy Route: {route}')
print(f'Total Distance: {"{:.1f}".format(total_dist)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

Greedy Route: ['Phoenix', 'Fort Worth', 'Memphis', 'Atlanta', 'Raleigh', 'Detroit', 'Miami', 'Portland', 'Phoenix']
Total Distance: 6408.1 nm
Calculation Time: 0.0 seconds


**Exact Shortest Path**

Brute force calculation for the shortest path, visiting all nodes once, and returning home.

In [17]:
t0 = time.time()
route, total_dist = get_exact_rte(home, dest_cities, weights)
t1 = time.time()

print(f'Shortest Route: {route}')
print(f'Total Distance: {"{:.1f}".format(total_dist)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

Shortest Route: ['Phoenix', 'Portland', 'Detroit', 'Raleigh', 'Miami', 'Atlanta', 'Memphis', 'Fort Worth', 'Phoenix']
Total Distance: 5585.2 nm
Calculation Time: 0.005983114242553711 seconds


**Heuristic Approach**

Using 2-opt swapping, we'll optimize a randomly generated Hamiltonian route from the source node.

In [18]:
t0 = time.time()
route, total_dist = get_random_rte(home, dest_cities, weights)
t1 = time.time()

print(f'Random Route: {route}')
print(f'Total Distance: {"{:.1f}".format(total_dist)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

t0 = time.time()
new_route, new_dist = get_2_opt_rte(route, weights)
t1 = time.time()
print(f'Heuristic Route: {new_route}')
print(f'Total Distance: {"{:.1f}".format(new_dist)} nm')
print(f'Calculation Time: {t1 - t0} seconds')

Random Route: ['Phoenix', 'Atlanta', 'Portland', 'Miami', 'Detroit', 'Memphis', 'Raleigh', 'Fort Worth', 'Phoenix']
Total Distance: 9417.3 nm
Calculation Time: 0.0 seconds
Heuristic Route: ['Phoenix', 'Atlanta', 'Miami', 'Raleigh', 'Detroit', 'Memphis', 'Fort Worth', 'Portland', 'Phoenix']
Total Distance: 6161.5 nm
Calculation Time: 0.0 seconds
